In [1]:
import pandas as pd
import numpy as np
import requests
# from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

In [2]:
df = pd.read_csv('Canada_geospatial.csv', header=0)
df.shape

(103, 5)

In [3]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
p_lat = df.iloc[1, 3]
p_lon = df.iloc[1, 4]

radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, p_lat, p_lon, VERSION, radius, LIMIT)

In [6]:
results = requests.get(url).json()

In [7]:
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c23d3aaf7ced13a5ed7216d',
   'name': 'Royal Canadian Legion',
   'location': {'address': '45 Lawson Rd',
    'lat': 43.78253332838298,
    'lng': -79.16308473261682,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.78253332838298,
      'lng': -79.16308473261682}],
    'distance': 304,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['45 Lawson Rd', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4c23d3aaf7ced13a5ed7216d-0'},
 {'reasons': {'count': 0,
   'items'

In [8]:
# Acquire venue information for all the postcodes from Foursquare
radius = 500
LIMIT = 100
postcode_list = []
postcode_lat_list = []
postcode_lon_list = []
venue_list = []
venue_lat_list = []
venue_lon_list = []
venue_category_list = []

for i in range(103):
    postcode_lat = df.iloc[i, 3]
    postcode_lon = df.iloc[i, 4]
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
    .format(CLIENT_ID, CLIENT_SECRET, postcode_lat, postcode_lon, VERSION, radius, LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for v in results:
        postcode_list.append(df.iloc[i, 0])
        postcode_lat_list.append(postcode_lat)
        postcode_lon_list.append(postcode_lon)
        venue_list.append(v['venue']['name'])
        venue_lat_list.append(v['venue']['location']['lat'])
        venue_lon_list.append(v['venue']['location']['lng'])
        venue_category_list.append(v['venue']['categories'][0]['name'])
        


toronto_venue = pd.DataFrame({'Postcode': postcode_list, 
                              'Postcode Latitude': postcode_lat_list, 
                              'Postcode Longtitude': postcode_lon_list, 
                              'Venue': venue_list, 
                              'Venue Latitude': venue_lat_list, 
                              'Venue Longitude': venue_lon_list,
                              'Venue Category': venue_category_list})

toronto_venue.head()

,Postcode,Postcode Latitude,Postcode Longtitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,M1B,43.806686,-79.194353,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,M1C,43.784535,-79.160497,Royal Canadian Legion,Bar,43.782533,-79.163085
2,M1C,43.784535,-79.160497,Scarborough Historical Society,History Museum,43.788755,-79.162438
3,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,Pizza Place,43.767697,-79.189914
4,M1E,43.763573,-79.188711,G & G Electronics,Electronics Store,43.765309,-79.191537


In [9]:
toronto_venue.shape

(2232, 7)

In [10]:
toronto_venue.to_csv('Toronto_Venue.csv', index=False)

In [5]:
# toronto_venue = pd.read_csv('Toronto_Venue.csv', header=0)

In [6]:
# Convert the category to numeric values
toronto_category = pd.get_dummies(toronto_venue['Venue Category'])
category_list = list(toronto_category.columns)
toronto_category['Postcode'] = toronto_venue['Postcode']

In [7]:
toronto_grouped = toronto_category.groupby('Postcode').mean().reset_index()

In [8]:
# k Means clustering
k = 5
toronto_grouped_k = toronto_grouped.drop('Postcode', axis=1)

kmeans = KMeans(n_clusters=k, random_state=4).fit(toronto_grouped_k)

In [9]:
kmeans.labels_[:5]

array([3, 0, 0, 0, 0])

In [10]:
len(kmeans.labels_)

100

In [11]:
# Merge label with location data
toronto_label = pd.DataFrame({'Postcode': toronto_grouped['Postcode'],
                              'Cluster': kmeans.labels_})
toronto_merged = pd.merge(df, toronto_label)
toronto_merged.shape

(100, 6)

In [12]:
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0


In [18]:
latitude = df.Latitude.mean()
longitude = df.Longitude.mean()
color_map = ['#FF5858', '#B95DF3', '#FFC158', '#629CF3', '#FFFD58']

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lon, poi, cluster in zip(toronto_merged.Latitude, toronto_merged.Longitude, toronto_merged.Postcode, toronto_merged.Cluster):
    label = folium.Popup(str(poi)+'\n Cluster: '+str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                        radius=5,
                        popup=label,
                        color=color_map[cluster],
                        fill=True,
                        fill_color=color_map[cluster],
                        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [13]:
toronto_grouped.head()

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
toronto_grouped.shape

(100, 276)

In [34]:
# Create the dataframe that display the top 5 venues for each postcode.
num_top_venues = 5

columns = ['Postcode']
for i in range(num_top_venues):
    columns.append('Top {} Most Common Venue'.format(i+1))

# toronto_grouped.iloc[15, :][1:].sort_values(ascending=False).index.values[:5]

In [82]:
postcode_list = []
Top_list = []

for i in range(100):
    postcode_list.append(toronto_grouped.iloc[i, 0])
    sorting = toronto_grouped.iloc[i, 1:].sort_values(ascending=False)
    # Check if there are actually 5 top venues or not  
    sorting = sorting[sorting>0]
    if len(sorting)>4:
        Top_list = Top_list+list(sorting.index)[:num_top_venues]
    else:
        index = list(sorting.index)+[np.nan]*(5-len(sorting))
        Top_list = Top_list+index

toronto_sorted = pd.DataFrame({columns[0]: postcode_list, 
                               columns[1]: Top_list[::5], 
                               columns[2]: Top_list[1::5], 
                               columns[3]: Top_list[2::5], 
                               columns[4]: Top_list[3::5], 
                               columns[5]: Top_list[4::5]})
toronto_sorted.head()

,Postcode,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
0,M1B,Fast Food Restaurant,NaN,NaN,NaN,NaN
1,M1C,History Museum,Bar,NaN,NaN,NaN
2,M1E,Pizza Place,Electronics Store,Rental Car Location,Mexican Restaurant,Medical Center
3,M1G,Coffee Shop,Korean Restaurant,NaN,NaN,NaN
4,M1H,Bakery,Hakka Restaurant,Bank,Thai Restaurant,Athletics & Sports


In [83]:
toronto_merged = pd.merge(toronto_merged, toronto_sorted, on='Postcode')

In [85]:
# Cluster 0
toronto_merged[toronto_merged.Cluster==0]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,History Museum,Bar,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,Pizza Place,Electronics Store,Rental Car Location,Mexican Restaurant,Medical Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bakery,Hakka Restaurant,Bank,Thai Restaurant,Athletics & Sports
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,Discount Store,Department Store,Bus Station,Coffee Shop,Chinese Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Fast Food Restaurant,Intersection,Bus Station
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Motel,Movie Theater,NaN,NaN
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0,Café,General Entertainment,Skating Rink,College Stadium,NaN
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304,0,Indian Restaurant,Pet Store,Chinese Restaurant,Brewery,Latin American Restaurant
11,M1R,Scarborough,"Maryvale,Wexford",43.750071,-79.295849,0,Breakfast Spot,Auto Garage,Bakery,Smoke Shop,Sandwich Place


In [86]:
# Cluster 1
toronto_merged[toronto_merged.Cluster==1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
14,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,1,Playground,Park,NaN,NaN,NaN
21,M2P,North York,York Mills West,43.752758,-79.400049,1,Park,Bank,NaN,NaN,NaN
38,M4J,East York,East Toronto,43.685347,-79.338106,1,Park,Metro Station,Convenience Store,NaN,NaN
42,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,NaN,NaN
48,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,NaN,NaN
62,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,NaN
71,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1,Field,Park,Hockey Arena,Trail,NaN
77,M6L,North York,"Maple Leaf Park,North Park,Upwood Park",43.713756,-79.490074,1,Basketball Court,Bakery,Park,Construction & Landscaping,NaN
88,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944,1,Park,Pool,River,NaN,NaN
97,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,1,Pizza Place,Park,Bus Line,NaN,NaN


In [87]:
# Cluster 2
toronto_merged[toronto_merged.Cluster==2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Convenience Store,NaN,NaN,NaN
95,M9N,York,Weston,43.706876,-79.518188,2,Convenience Store,NaN,NaN,NaN,NaN


In [88]:
# Cluster 3
toronto_merged[toronto_merged.Cluster==3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3,Fast Food Restaurant,NaN,NaN,NaN,NaN
23,M3A,North York,Parkwoods,43.753259,-79.329656,3,Fast Food Restaurant,Bus Stop,Park,Food & Drink Shop,NaN
28,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,3,Airport,Park,Bus Stop,NaN,NaN
72,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,3,Park,Fast Food Restaurant,Market,Pharmacy,Women's Store


In [89]:
# Cluster 4
toronto_merged[toronto_merged.Cluster==4]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Top 1 Most Common Venue,Top 2 Most Common Venue,Top 3 Most Common Venue,Top 4 Most Common Venue,Top 5 Most Common Venue
19,M2L,North York,"Silver Hills,York Mills",43.75749,-79.374714,4,Cafeteria,NaN,NaN,NaN,NaN
